# Исследование надёжности заёмщиков

**Содержание**
1. [Описание проекта](#desc)
2. [Обзор данных](#review)
3. [Предобработка данных](#pred)
4. [Анализ данных](#analiz)
5. [Итоговый вывод](#it)

<a id="desc"></a> 
## Описание проекта

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Цель исследования** — ответить на 4 вопроса:

1. Есть ли зависимость между наличием детей и возвратом кредита в срок?
2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
4. Как разные цели кредита влияют на его возврат в срок?

**Ход исследования**

Исследование пройдёт в три этапа:

1. Обзор данных.
2. Предобработка данных.
3. Исследование.

<a id="review"></a> 
## Обзор данных

In [1]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
df.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

Итак, в таблице двенадцать столбцов. Тип данных в столбцах:

`days_employed`, `total_income` - float64;
`children`,`dob_years`,`education_id`,`family_status_id`,`debt` - int64
`education`,`family_status`,`gender`,`income_type`,`purpose` - object.

Согласно документации к данным:

`children` — количество детей в семье;

`days_employed` — общий трудовой стаж в днях;

`dob_years` — возраст клиента в годах;

`education` — уровень образования клиента;

`education_id` — идентификатор уровня образования;

`family_status` — семейное положение;

`family_status_id` — идентификатор семейного положения;

`gender` — пол клиента;

`income_type` — тип занятости;
`debt` — имел ли задолженность по возврату кредитов: со значением 1 - имел долг, 0 - не имел;

`total_income` — ежемесячный доход;

`purpose` — цель получения кредита.

В каждой строке таблицы - данные о клиенте банка. Названия колонок написаны в правильном стиле. Но количество значений в столбцах различается. Значит, в данных есть пропущенные значения. Также в колонке **education** я заметила дубликаты, а в колонке **days_employed** есть отрицательные значения. И еще в колонках **days_employed** и **total_income** много цифр после запятой. Прежде чем начать проверять гипотезы, нужно глубже исследовать данные и устранить проблемы в них.

<a id="pred"></a> 
## Предобработка данных

### Обработка пропусков

In [3]:
df.isna().sum()


children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Устраним пропуски в столбце `total_income`. Для этого создадим сначала небольшой датасет, где будет подсчитана медианная зарплата для каждой группы `income_type`.

In [4]:
median_income_group = df.groupby('income_type')['total_income'].median()
total_income_median = pd.DataFrame({'total_income_median':median_income_group}).reset_index()
total_income_median 

,income_type,total_income_median
0,безработный,131339.751676
1,в декрете,53829.130729
2,госслужащий,150447.935283
3,компаньон,172357.950966
4,пенсионер,118514.486412
5,предприниматель,499163.144947
6,сотрудник,142594.396847
7,студент,98201.625314


Добавим к таблице столбец с медианной зарплатой, соответствующей определенному типу занятости.

In [5]:
df = df.merge(total_income_median, on='income_type', how='left')
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_median
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,142594.396847
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,142594.396847
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,142594.396847
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,142594.396847
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,118514.486412


Заменим пропущенные значения в столбце `total_income` медианной зарплатой.

In [6]:
df['total_income'] = df['total_income'].fillna(value=df['total_income_median'])
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_median
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,142594.396847
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,142594.396847
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,142594.396847
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,142594.396847
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,118514.486412


Пропуски в столбце `days_employed` не так важны для нашей работы, поэтому их можно просто заменить явными обозначениями.

In [7]:
df['days_employed'] = df['days_employed'].fillna(value=0)
df.isna().sum()

children               0
days_employed          0
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
total_income_median    0
dtype: int64

**Вывод**

Пропуски есть в колонках `days_employed` и `total_income`. Причем, в столбце `days_employed` есть также отрицательные значения (скорее всего, это или опечатки, или какой-то сбой программы при переносе данных). Если учесть то, что количество пропусков в обеих колонках одинаково, я предполагаю, что эти значения связаны друг с другом расчетом (одно получается из другого, например). На практике было бы правильно установить причину отрицательных значений и пропусков и восстановить данные. Я бы обратилась к "хозяину" данных и уточнила эти вопросы. Но поскольку такой возможности нет, то:

1) Данные в колонке `total_income` важны для исследования, поэтому просто удалить эти строки нельзя. Пропуски можно заменить медианным значением. Можно было бы заменить и средним арифметическим, но поскольку эти данные обозначают доходы клиента, то здесь могут быть выбросы, соответственно, среднее значение может быть искаженным.

2) Данные в колонке `days_employed` не влияют на исследование, поэтому их можно просто заменить явным обозначением (например, '0').

3) Поскольку данные в столбце `days_employed` не нужны для дальнейших расчетов, я решила ничего не делать с отрицательными значениями.

### Замена типа данных

In [8]:
df['total_income'] = df['total_income'].astype('int')
df['days_employed'] = df['days_employed'].astype('int')
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_median
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,142594.396847
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,142594.396847
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,142594.396847
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,142594.396847
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,118514.486412


**Вывод**

Колонки`days_employed` (общий трудовой стаж в днях) и `total_income` (ежемесячный доход) имели тип данных float64. Для дальнейшего анализа и более читабельных и "красивых" результатов данные этих колонок лучше перевести в целочисленный тип.

### Обработка дубликатов

Проверим столбцы на дубликаты и "необычные" значения.

In [9]:
display(df['children'].unique())
display(df['education'].unique())
display(df['family_status'].unique())
display(df['income_type'].unique())
df['gender'].unique()


array([ 1,  0,  3,  2, -1,  4, 20,  5])

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

array(['F', 'M', 'XNA'], dtype=object)

In [10]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [11]:
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [12]:
df = df.drop(df[df['children'] == -1].index)


In [13]:
df = df.drop(df[df['children'] == 20].index)

In [14]:
df = df.drop(df[df['gender'] == 'XNA'].index)

In [15]:
df['children'].value_counts()

0    14148
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

In [16]:
df['gender'].value_counts()

F    14154
M     7247
Name: gender, dtype: int64

In [17]:
df['education'] = df['education'].str.lower()
df['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Найдем явные дубликаты и избавимся от них.

In [18]:
df.duplicated().sum()

71

In [19]:
df = df.drop_duplicates().reset_index(drop=True)

In [20]:
df.duplicated().sum()

0

**Вывод**
    
* В столбце `children` были строки с 2 аномальными значениями: **-1** и **20**. Доля их очень мала: 0,21% и 0,35% соответственно. Поэтому строки с ними можно удалить.
* Также в столбце `gender` я обнаружила странное значение **'XNA'**. Во всей таблице только одна строка с таким значением, поэтому ее можно также удалить.
* Затем, применив метод **unique()** к столбцу `education`, я выяснила, что там есть названия, которые отличаются только лишь регистром нескольких букв. Применив метод str.lower(), я избавилась и от этих дупликатов. Возможная причина появления таких дупликатов - это отсутствие определенных правил при заполнении этого столбца. 
*Потом с помощью методов **duplicated()** и **drop_duplicates()** были удалены явные дупликаты.</div>


### Лемматизация

In [21]:
purpose_unique = df['purpose'].unique() # создадим список с уникальными значениями целей
purpose_unique

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [22]:
#импортируем библиотеку
from pymystem3 import Mystem
m = Mystem()

lemmas_list = [] # создадим пустой список для лемматизированных целей
for i in purpose_unique:
    lemmas_list.append(' '.join(m.lemmatize(i)))   


lemmas_list = ''.join(lemmas_list) # превратим список в строку для подсчета слов

from collections import Counter
Counter(m.lemmatize(lemmas_list))

Counter({'покупка': 10,
         '   ': 59,
         'жилье': 7,
         ' \n': 38,
         'приобретение': 1,
         'автомобиль': 9,
         'дополнительный': 2,
         'образование': 9,
         'сыграть': 1,
         'свадьба': 3,
         'операция': 4,
         'с': 5,
         'на': 4,
         'проведение': 1,
         'для': 2,
         'семья': 1,
         'недвижимость': 10,
         'коммерческий': 2,
         'жилой': 2,
         'строительство': 3,
         'собственный': 1,
         'подержать': 1,
         'свой': 4,
         'со': 1,
         'заниматься': 2,
         'сделка': 2,
         'подержанный': 1,
         'получение': 3,
         'высокий': 3,
         'профильный': 1,
         'сдача': 1,
         'ремонт': 1})

**Вывод**

Применив лемматизацию, мы можем выделить несколько ключевых целей кредита: жилье, автомобиль, свадьба, образование

### Категоризация данных

Создадим словари `education_dict` и `family_status_dict`.

In [23]:
education_dict = df[['education_id','education']]
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
education_dict

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [24]:
family_status_dict = df[['family_status_id','family_status']]
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)
family_status_dict

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


In [25]:
df.drop(['education', 'family_status', 'total_income_median'], axis = 1, inplace=True)

**Категоризация по столбцу "children"**

In [26]:
df.groupby('children')[['children']].count()

,children
children,
0,14090
1,4808
2,2052
3,330
4,41
5,9


Для того, чтобы было легче анализировать данные по детям, напишу функцию, которая объединит данные о наличии детей в 3 категории: **нет детей**, **1-2 ребенка**, **многодетные**.

In [27]:
def children_group(children):
    #"""
    #Возвращает группу по наличию детей `children`, используя правила:
    #- 'нет детей', если children = 0;
    #- '1-2 ребенка', если children = 1 или 2;
    #- 'многодетные' — если children >= 3.
    #"""
    
    if children == 0:
        return 'нет детей'
    if (children == 1) or (children == 2):
        return '1-2 ребенка'
    return 'многодетные' 

Применим эту функцию для всех значений столбца `children` и запишем результат ее работы в новый столбец `children_group`.

In [28]:
df['children_group'] = df['children'].apply(children_group)
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,children_group
0,1,-8437,42,0,0,F,сотрудник,0,253875,покупка жилья,1-2 ребенка
1,1,-4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,1-2 ребенка
2,0,-5623,33,1,0,M,сотрудник,0,145885,покупка жилья,нет детей
3,3,-4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,многодетные
4,0,340266,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,нет детей


**Категоризация по столбцу "family_status_id"**

In [29]:
df['family_status_id'].value_counts()

0    12261
1     4133
4     2796
3     1189
2      951
Name: family_status_id, dtype: int64

Напишем функцию, которая которая объединит данные о семейном положении в 2 группы.

In [30]:
def family_status_group(status):
    #"""
    #Возвращает группу по семейному положению `family_status`, используя правила:
    #- 'Состоит в браке', если family_status = женат / замужем или гражданский брак;
    #- 'Не состоит в браке', если family_status = Не женат / не замужем, в разводе или вдовец / вдова;
    #"""
    
    if (status == 0) or (status == 1):
        return 'В браке'
    return 'Не состоит в браке' 

Применим эту функцию для всех значений столбца `family_status_id` запишем результат ее работы в новый столбец `family_status_group`.

In [31]:
df['family_status_group'] = df['family_status_id'].apply(family_status_group)
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,children_group,family_status_group
0,1,-8437,42,0,0,F,сотрудник,0,253875,покупка жилья,1-2 ребенка,В браке
1,1,-4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,1-2 ребенка,В браке
2,0,-5623,33,1,0,M,сотрудник,0,145885,покупка жилья,нет детей,В браке
3,3,-4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,многодетные,В браке
4,0,340266,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,нет детей,В браке


**Категоризация по столбцу "total_income"**

Поделим данные столбца "total_income" на 4 группы и посмотрим, какие интервалы получатся.

In [32]:
pd.qcut(df['total_income'], q=5)

0        (214605.0, 2265604.0]
1          (98514.0, 132113.0]
2         (132113.0, 161379.4]
3        (214605.0, 2265604.0]
4         (132113.0, 161379.4]
                 ...          
21325    (214605.0, 2265604.0]
21326     (132113.0, 161379.4]
21327     (20666.999, 98514.0]
21328    (214605.0, 2265604.0]
21329     (20666.999, 98514.0]
Name: total_income, Length: 21330, dtype: category
Categories (5, interval[float64]): [(20666.999, 98514.0] < (98514.0, 132113.0] < (132113.0, 161379.4] < (161379.4, 214605.0] < (214605.0, 2265604.0]]

Создадим новый столбец `total_income_group`, где будет прописана группа в зависимости от уровня дохода.

In [33]:
names = ['очень низкий', 'низкий', 'средний', 'высокий', 'очень высокий'] # создадим список с названиями категорий по уровню дохода 
df['total_income_group'] = pd.qcut(df['total_income'], q=5, labels=names)
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,children_group,family_status_group,total_income_group
0,1,-8437,42,0,0,F,сотрудник,0,253875,покупка жилья,1-2 ребенка,В браке,очень высокий
1,1,-4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,1-2 ребенка,В браке,низкий
2,0,-5623,33,1,0,M,сотрудник,0,145885,покупка жилья,нет детей,В браке,средний
3,3,-4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,многодетные,В браке,очень высокий
4,0,340266,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,нет детей,В браке,средний


**Категоризация по столбцу "purpose"**

Сначала напишем функцию для определения цели кредита.

In [34]:
def purpose_cat(purpose):
    #"""
    #Возвращает группу в зависимости от цели кредита `purpose`, используя правила:
    #- 'автомобиль', если в столбце `purpose` есть ключевое слово "авто";
    #- 'недвижимость', если в столбце `purpose` есть ключевые слова "недвижимость" или "жилье";
    #- 'свадьба', если в столбце `purpose` есть ключевое слово "свадьба";
    #- 'образование', если в столбце `purpose` есть ключевое слово "образование";
    #"""
    lemma = ' '.join(m.lemmatize(purpose))
 
    if 'авто' in lemma:
        return 'автомобиль'
    if ('недвиж' in lemma) or ('жил' in lemma):
        return  'недвижимость'
    if 'свадьба' in lemma:
        return  'свадьба'
    if 'образов' in lemma:
        return 'образование'
    else:
        return 'другое'

Применим эту функцию к столбцу `purpose`, и создадим новый столбец `purpose_group`, где для каждого клиента будет прописана группа по цели кредита.

In [35]:
df['purpose_group'] = df['purpose'].apply(purpose_cat)
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,children_group,family_status_group,total_income_group,purpose_group
0,1,-8437,42,0,0,F,сотрудник,0,253875,покупка жилья,1-2 ребенка,В браке,очень высокий,недвижимость
1,1,-4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,1-2 ребенка,В браке,низкий,автомобиль
2,0,-5623,33,1,0,M,сотрудник,0,145885,покупка жилья,нет детей,В браке,средний,недвижимость
3,3,-4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,многодетные,В браке,очень высокий,образование
4,0,340266,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,нет детей,В браке,средний,свадьба


**Вывод**

В нашей таблице были столбцы, которые не нужны по сути для расчетов. Кроме того, были столбцы-дубли, с "id" и их расшифровкой (это столбцы `education_id` и `education`, `family_status_id` и `family_status`) Для оптимизации расчетов, расшифровки вынесли в отдельные словари. А в основной таблице оставили "id" и нужные для расчетов данные. Также для дальнейшего исследования я провела категоризацию в 4 колонках.
* колонка "children": группы - **нет детей**, **1-2 ребенка** и **многодетные**.
* колонка "family_status_id": группы - **в браке** и **не состоит в браке**.
* колонка "total_income": группы - **очень низкий**, **низкий**, **средний**, **высокий**, **очень высокий**.
* колонка "purpose": группы - **автомобиль**, **недвижимость**, **свадьба** и **образование**.

<a id="analiz"></a> 
## Анализ данных

**ОТВЕТИМ НА ВОПРОСЫ**

**- Есть ли зависимость между наличием детей и возвратом кредита в срок?**

Выясним, какова доля должников по кредитам в каждой из 3 групп: **нет детей**, **1-2 ребенка** и **многодетные**. Для этого для каждой группы нужно количество должников поделить на количество всех людей в группе.

In [36]:
children_grouped_final = df.groupby(['children_group']).agg({'debt':['count', 'sum']})
children_grouped_final['share'] = children_grouped_final['debt']['sum'] / children_grouped_final['debt']['count']  
children_grouped_final.sort_values(by='share', ascending=False)

debt           share
                count   sum          
children_group                       
1-2 ребенка      6860   638  0.093003
многодетные       380    31  0.081579
нет детей       14090  1063  0.075444

**Вывод**

Итак, самая высокая доля должников в группе **"1-2 ребенка"**, а самая низкая - в группе **"нет детей"**.

**- Есть ли зависимость между семейным положением и возвратом кредита в срок?**

Выясним, какова доля должников по кредитам в каждой из 2 групп: **в браке** и **не состоит в браке**.

In [37]:
family_status_grouped = df.groupby(['family_status_group']).agg({'debt':['count', 'sum']})
family_status_grouped['share'] = family_status_grouped['debt']['sum'] / family_status_grouped['debt']['count']  
family_status_grouped

debt           share
                     count   sum          
family_status_group                       
В браке              16394  1312  0.080029
Не состоит в браке    4936   420  0.085089

**Вывод**

Доля должников в группах  одинаковая - 8%.

**- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

Выясним, какова доля должников по кредитам в каждой из 5 групп.

In [38]:
total_income_grouped = df.groupby(['total_income_group']).agg({'debt':['count', 'sum']})
total_income_grouped['share'] = total_income_grouped['debt']['sum'] / total_income_grouped['debt']['count']  
total_income_grouped

debt          share
                   count  sum          
total_income_group                     
очень низкий        4267  344  0.080619
низкий              4266  358  0.083919
средний             4265  373  0.087456
высокий             4266  358  0.083919
очень высокий       4266  299  0.070089

**Вывод**

Итак, меньше всего доля должников в группе по доходам "**очень высокий**", тогда как в остальных группах доля должников чуть больше и примерно одинакова.

**- Как разные цели кредита влияют на его возврат в срок?**

Выясним, какова доля должников по кредитам в каждой группе: **автомобиль**, **образование**, **недвижимость** и **свадьба**.

In [39]:
purpose_grouped = df.groupby(['purpose_group']).agg({'debt':['count', 'sum']})
purpose_grouped['share'] = purpose_grouped['debt']['sum'] / purpose_grouped['debt']['count']  
purpose_grouped.sort_values(by='share', ascending=False)

debt          share
               count  sum          
purpose_group                      
автомобиль      4279  400  0.093480
образование     3988  369  0.092528
свадьба         2313  183  0.079118
недвижимость   10750  780  0.072558

**Вывод**

Из таблицы видно, что доля должников в группах с целями **"автомобиль"** и **"образование"** выше, чем в группах с целями **"недвижимость"** и **"свадьба"**.

<a id="it"></a> 
## Общий вывод

Итак, изучив входные данные, я выяснила, то в данных есть пропуски в нескольких столбцах,также обнаружились дубликаты в столбце "education" ("образование") и аномальные значения в колонках "gender" ("пол") и "children" ("количество детей"). Проблемы в данных я устранила. Часть строк пришлось удалить, но поскольку их доля очень мала, на исследование это не повлияло.
В исследовании я ответила на 4 вопроса:
1. Есть ли зависимость между наличием детей и возвратом кредита в срок?
Есть, но небольшая - самая высокая доля должников в группе **"1-2 ребенка"**, а самая низкая - в группе **"нет детей"**.
2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
Нет, доля должников в обеих группах - 8%.
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
Незначительная зависимость есть. Меньше всего доля должников в группе по доходам **"очень высокий"**.
4. Как разные цели кредита влияют на его возврат в срок?
Доля должников в группах с целями **"автомобиль"** и **"образование"** чуть больше, чем в группах с целями **"недвижимость"** и **"свадьба"**.

В целом, сильной зависимости возврата кредита в срок от этих 4 параметров нет. Разница между параметрами в группах около 1 - 2 %, что очень мало.